Logistic Regression to predict if songs contain love

In [27]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pandas as pd
import seaborn as sns


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [28]:
#Read in datasets
spotify_data = pd.read_csv("spotify_data.csv")

In [29]:
#Adding Has Love column
song_attribute_data = spotify_data

titles = song_attribute_data["Song"]
titles = titles.str.split(pat=" ", expand=True)

song_attribute_data["Has Word Love?"] = song_attribute_data['Song'].str.contains("Love")
song_attribute_data_love = song_attribute_data[song_attribute_data["Has Word Love?"]]

song_attribute_data = song_attribute_data.dropna()


In [30]:
#Prepare X and y
y = song_attribute_data["Has Word Love?"]
X = song_attribute_data.loc[:,["speechiness", "valence", "tempo", "time_signature", "Popularity Points Awarded"]]

def minmax(z):
    z = (z-min(z))/(max(z)-min(z))
    return z
X = X.apply(minmax,axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2, random_state=100)

In [31]:
#Run Regression
reg = LogisticRegression(penalty = None,
                         fit_intercept=True,
                         solver = 'newton-cholesky',
                         max_iter=1000).fit(X_train,y_train)

print("Training R^2" , reg.score(X_train, y_train))
print("Test R^2", reg.score(X_test, y_test))
#Pretty good accuracy scores for these predictors


Training R^2 0.9174862912350944
Test R^2 0.9136790810998956


In [46]:
#Run regression with all variables
X_n = song_attribute_data.iloc[:, [8] + [9] + list(range(11, 24))]
X_n = X_n.apply(minmax,axis=1)

dummies = pd.DataFrame([])
new_dummies = pd.get_dummies(song_attribute_data.loc[:,"spotify_track_explicit"], drop_first=True, dtype=int)
dummies = pd.concat([dummies, new_dummies], axis=1, ignore_index=True)

X2 = pd.concat([X_n,dummies],axis=1)
X2.columns = X2.columns.astype(str)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2,y, test_size=.2, random_state=100)

reg2 = LogisticRegression(penalty = None,
                         fit_intercept=True,
                         solver = 'newton-cholesky',
                         max_iter=1000).fit(X2_train,y2_train)

print("Training R^2", reg2.score(X2_train, y2_train))
print("Test R^2", reg2.score(X2_test, y2_test))
#Exact same R^2 values?

Training R^2 0.9174862912350944
Test R^2 0.9136790810998956
